In [13]:
from pymongo import MongoClient
from pymongo.errors import OperationFailure
from os import getenv
from dotenv import load_dotenv
from datetime import datetime
import ssl
from bson.json_util import loads as bson_loads, dumps as bson_dumps 
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [2]:
import etherscan
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt # importing matplotlib
import numpy as np # importing numpy
import schedule
import time
import json
import requests

from dotenv import load_dotenv
from datetime import datetime
from web3 import Web3
from os import getenv
import os
from tqdm import tqdm
from decimal import Decimal

In [8]:
# address book
carlisle = '0xd791427e372186E90637caF1D6D6AFb054BeeC7A'
rarible_contract = '0xcd4EC7b66fbc029C116BA9Ffb3e59351c20B5B06'
opensea = '0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b'
foundation = '0xcDA72070E455bb31C7690a170224Ce43623d0B6f'
superrare = '0x41A322b28D0fF354040e2CbC676F0320d8c8850d'
knownorigin = '0x848b0EA643e5a352D78e2c0c12A2dD8c96feC639'

In [3]:
load_dotenv()

# signing on to the MongoDB and viewing currents DBs
client = MongoClient(getenv('MONGO'), ssl_cert_reqs=ssl.CERT_NONE)
print(client.list_database_names())  # output: ['admin', 'local']

['spirits', 'test', 'admin', 'local']


In [4]:
es = etherscan.Client(api_key=getenv('ETHERSCAN'))
w3 = Web3(Web3.HTTPProvider(getenv('INFURA')))

w3.isConnected()

True

In [33]:
# The DB we will be working on "test"
db = client.test

# initializing a collection for DBs
collection = db.col

In [5]:
user = {'name': 'Halil', 'lang': 'Python'}
insert_user = collection.insert_one(user)

print(insert_user.inserted_id)  # output: 5ed01eecd5e7afe4b2f2f7c9
print(client.list_database_names())  # output: ['test', 'admin', 'local']
print(db.list_collection_names())  # output: ['col']

608470b71d8e3f463993b76e
['spirits', 'test', 'admin', 'local']
['col']


In [ ]:
block = w3.eth.getBlock('latest')
block.transactions

In [44]:
# initializing a collection for DBs
transactions = db.ether

# content
post = {'difficulty': block.difficulty, 
        'extraData': block.extraData, 
        'hash': block.hash,
        'transactions': block.transactions,
        'post_date': datetime.now()}

# inseting content into collection
insert_post = transactions.insert_one(post)

# id of insert
print(insert_post.inserted_id)  

6085017e1d8e3f463993b772


In [46]:
transaction = '0x7a04337a420fede66c1038c3288b250c8e1ffd3fd6de46a3ec44a09150d803b8'

contract = w3.eth.getTransaction(transaction)
contract

AttributeDict({'blockHash': HexBytes('0xc190a41f539d01561b987161ee34f4cbf77112e3f5631974abb97db6bbdedd53'),
 'blockNumber': 12292666,
 'from': '0x81085713E6dB7b9511b85CE5ed33C790980BFA36',
 'gas': 203534,
 'gasPrice': 140000000000,
 'hash': HexBytes('0x7a04337a420fede66c1038c3288b250c8e1ffd3fd6de46a3ec44a09150d803b8'),
 'input': '0xa0712d6800000000000000000000000000000000000000000000000dcc988cf91e06d800',
 'nonce': 9,
 'r': HexBytes('0x2440d9482e3b6a7225300f2361057bb6b59c7cdfcb334546ac1e16738e5cb0f9'),
 's': HexBytes('0x23ce84bbdcc4f8fa1102bfaedad9c616a72c22626c7608b6b64b1c3b101634bd'),
 'to': '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643',
 'transactionIndex': 312,
 'type': '0x0',
 'v': 38,
 'value': 0})

In [48]:
# initializing a collection for DBs
transactions = db.transations

# content
post = {'blockHash': contract.blockHash, 
        'blockNumber': contract.blockNumber, 
        'gas': contract.gas,
        'gasPrice': contract.gasPrice,
        'hash': contract.hash,
        'input': contract.input,
        'post_date': datetime.now()}

# inseting content into collection
insert_post = transactions.insert_one(post)

# id of insert
print(insert_post.inserted_id) 

608502e01d8e3f463993b773


In [17]:
#erc20
ABI = json.loads('[{
"constant": true,
"inputs": [{"name": "_owner", "type": "address"}],
"name": "balanceOf",
"outputs": [{"name": "", "type": "uint256"}],
"payable": false,
"stateMutability": "view",
"type": "function"
}]')


SyntaxError: EOL while scanning string literal (<ipython-input-17-6b0686591ab9>, line 2)

In [10]:
wallet_address = carlisle
wallet_address = Web3.toChecksumAddress(wallet_address)

token_contract_address = '0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f'
token_contract_address = Web3.toChecksumAddress(token_contract_address)

# define contract
contract = w3.eth.contract(token_contract_address, abi=ABI)

# call contract and get data from balanceOf for argument wallet_address
raw_balance = contract.functions.balanceOf(wallet_address).call()

# convert the value from Wei to Ether
synthetix_value = Web3.fromWei(raw_balance, 'ether')

In [11]:

sample_transport=RequestsHTTPTransport(
   url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
   verify=True,
   retries=5,
)
client = Client(
   transport=sample_transport
)

# Get the value of SNX/ETH 
query = gql('''
query {
 pair(id: "0x43ae24960e5534731fc831386c07755a2dc33d47"){
     reserve0
     reserve1
}
}
''')
response = client.execute(query)
snx_eth_pair = response['pair']
eth_value = float(snx_eth_pair['reserve1']) / float(snx_eth_pair['reserve0'])

# Get the value of ETH/DAI
query = gql('''
query {
 pair(id: "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11"){
     reserve0
     reserve1
}
}
''')
response = client.execute(query)
eth_dai_pair = response['pair']
dai_value = float(eth_dai_pair['reserve0']) / float(eth_dai_pair['reserve1'])

snx_dai_value = eth_value * dai_value

In [12]:
snx_dai_value

14.95091843064734

In [4]:
if "test" in client.list_database_names():
    print("You successfully created test database.")
else:
    print("test database was not created.")

test database was not created.
